# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [161]:
import pandas as pd
df1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv')

In [163]:
def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
            .str.lower()
            .str.replace(r"\s", "_", regex=True)
            .str.replace(r"state", "st", regex=True)
    )
    return df

df1 = standardize_columns(df1)
df2 = standardize_columns(df2)
df3 = standardize_columns(df3)

In [165]:
combined_df = pd.concat([df1, df2, df3], ignore_index = True)
combined_df.drop_duplicates(inplace = True)
combined_df.reset_index(drop = True, inplace = True)

In [167]:
combined_df["gender"] = combined_df["gender"].str.strip().str.lower().replace({
    "male": "M",
    "m": "M",
    "female": "F",
    "femal": "F",
    "f": "F"
})

In [177]:
combined_df.rename(columns={"st": "state"}, inplace=True)

In [181]:
usa_states_dict = {
    "AZ": "Arizona", 
    "CA": "California", 
    "NV": "Nevada", 
    "WA": "Washington"
}

combined_df["state"] = combined_df["state"].apply(lambda state: usa_states_dict.get(str(state).strip().upper(), state) if pd.notna(state) else state)

combined_df.replace({
    "state": {
        "Cali": "California"
    },
    "education": {
        "Bachelors": "Bachelor"
    },
    "vehicle_class": {
        "Sports Car": "Luxury",
        "Luxury SUV": "Luxury",
        "Luxury Car": "Luxury"
    }   
}, inplace=True)

In [185]:
if combined_df["customer_lifetime_value"].dtype != "float":
    combined_df["customer_lifetime_value"] = combined_df["customer_lifetime_value"].str.replace("%", "", regex=False).astype(float)
    
combined_df["customer_lifetime_value"] = round(combined_df["customer_lifetime_value"], 2)

In [187]:
combined_df["number_of_open_complaints"] = combined_df["number_of_open_complaints"].apply(
    lambda complaint: int(str(complaint).split("/")[1]) if pd.notna(complaint) and "/" in str(complaint) else (int(complaint) if pd.notna(complaint) else pd.NA)
).astype("Int64")

In [191]:
combined_df["total_claim_amount"] = round(combined_df["total_claim_amount"], 2)

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [201]:
df4 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')
df4.drop("unnamed:_0", axis = 1)

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,...,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,1
10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,...,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,1
10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,...,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,A,2
10908,WA60547,California,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,...,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2


In [227]:
df4["sales_channel"].unique()
summary2 = df4.pivot_table(values = "total_claim_amount", index = "sales_channel", aggfunc = "sum").sort_values(by = "total_claim_amount", ascending = False).reset_index()
summary2["total_claim_amount"] = round(summary2["total_claim_amount"], 2)
summary2

,sales_channel,total_claim_amount
0,Agent,1810226.82
1,Branch,1301204.00
2,Call Center,926600.82
3,Web,706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [229]:
summary = df4.pivot_table(values = "customer_lifetime_value", index = ["gender", "education"], aggfunc = "mean").sort_values(by = "customer_lifetime_value", ascending = False).reset_index()
summary["customer_lifetime_value"] = round(summary["customer_lifetime_value"], 2)
summary

,gender,education,customer_lifetime_value
0,F,High School or Below,8675.22
1,M,Master,8168.83
2,F,Master,8157.05
3,M,High School or Below,8149.69
4,M,College,8052.46
5,F,Bachelor,7874.27
6,F,College,7748.82
7,M,Bachelor,7703.60
8,M,Doctor,7415.33
9,F,Doctor,7328.51


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here